<a href="https://colab.research.google.com/github/gab-palmeri/aml-geolocalization/blob/develop/step_4_reranking_geowarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to train the model with arcface and sphereface loss function and test with our databases.

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [1]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"
!pip3 install "kornia"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 27.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 21.7 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=c5904a8c59c44936388c77ea9d152aa2efb5b6cef42a717a656cf5b94b048f5d
  Stored in directory: /root/.cache/pip/wheels/65/e2/d8/878a8cc986641056fbfebefc4d8eb64238a7b6d3426e86b447
Successfully built utm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 36.1 MB/s 
cuda:0


# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [4]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


use_mount = True
resume_model = True

if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

if not os.path.isdir("/content/tokyo_xs"):
  raise FileNotFoundError(f"Can't download tokyo xs")

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo-night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1EZJY2r5565-iVk2oEbTns0xc4F_em4Y4"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

if not os.path.isdir("/content/tokyo-night"):
  raise FileNotFoundError(f"Can't download tokyo night")

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")

if resume_model and not os.path.isfile("/content/saved_models/cosface_model.pth"):
  if use_mount:
    !mkdir "/content/saved_models"
    !cp "/content/drive/MyDrive/step_4_rerank/cosface_model.pth" "/content/saved_models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/saved_models’: File exists


# Download Code

Clone of original repo of CosPlace and our code

In [8]:
# download code of CosPlace
!git clone "https://github.com/gmberton/CosPlace" 
#!rm -r "/content/CosPlace"

# download code of GeoWarp
!git clone "https://github.com/gmberton/geo_warp"

# download our code
#!rm -rf Team
!git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
!mv "/content/aml-geolocalization/" "/content/Team"
#!rm -r "/content/aml-geolocalization"

fatal: destination path 'CosPlace' already exists and is not an empty directory.
fatal: destination path 'geo_warp' already exists and is not an empty directory.
Cloning into 'aml-geolocalization'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 151 (delta 24), reused 51 (delta 22), pack-reused 96
Receiving objects: 100% (151/151), 1.96 MiB | 40.04 MiB/s, done.
Resolving deltas: 100% (53/53), done.




# Import Code


In [3]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/geo_warp/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

import Team.GeoWarp as GeoWarp
from GeoWarp import *
from Team import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [4]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Drive related functions to save and load training checkpoints

In [5]:
import os
from google.colab import drive

drive.mount('/content/drive')

def drive_save_checkpoint(output_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !zip -r "/content/drive/MyDrive/project6/{time}.zip" "/content/{output_folder}"

def drive_load_checkpoint(input_folder, time):
    drive_folder = "/content/drive/MyDrive/project6/"
    !jar xvf "/content/drive/MyDrive/project6/{time}.zip"

def drive_tester(output_folder):
    drive_folder = "/content/drive/MyDrive/project6/"
    if not os.path.exists(drive_folder):
        !mkdir "/content/drive/MyDrive/project6/"
    !touch "/content/drive/MyDrive/project6/test"

    if os.path.exists("/content/drive/MyDrive/project6/test"):
        logging.info("Drive saving works")
    else:
        logging.info("WARNING: Drive saving does not work")

# use this function when content on drive are not updated
def drive_refresh():
  drive.flush_and_unmount()
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parameters

Original code provides two main executable files: `train.py` and `eval.py`. We want to reproduce the same behaviour of these two files so these are the parameters passed via terminal. They will be put in a dict called `args` to simply reuse code inside ex files

### Save dir for CosFace

In [6]:
saveDirLocal = "cosface"

## Setup parameters

In [7]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "resnet18"   # ["vgg16", "resnet18", "resnet50", "resnet101", "resnet152"]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
# Training parameters
USE_AMP_16 = "store_true"       # use Automatic Mixed Precision
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = None     # Path to model to resume training from
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets
SAVEDIR = saveDirLocal


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")


# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp_16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER, 'save_dir': SAVEDIR,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder,
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


# Setup logging

In [8]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

INFO:root:/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-c2f10f1c-54ca-424c-8b16-53a564ce741a.json
INFO:root:Arguments: {'M': 10, 'alpha': 30, 'N': 5, 'L': 2, 'groups_num': 1, 'min_images_per_class': 10, 'backbone': 'resnet18', 'fc_output_dim': 512, 'use_amp_16': 'store_true', 'augmentation_device': 'cuda', 'batch_size': 32, 'epochs_num': 3, 'iterations_per_epoch': 10000, 'lr': 1e-05, 'classifiers_lr': 0.01, 'brightness': 0.7, 'contrast': 0.7, 'hue': 0.5, 'saturation': 0.7, 'random_resized_crop': 0.5, 'infer_batch_size': 16, 'positive_dist_threshold': 25, 'resume_train': None, 'resume_model': None, 'device': 'cuda', 'seed': 0, 'num_workers': 8, 'dataset_folder': '/content/small', 'save_dir': 'cosface', 'val_set_folder': '/content/small/val', 'train_set_folder': '/content/small/train'}
INFO:root:The outputs are being saved in logs/cosface/2022-12-27_17-38-42


# PAY ATTENTION!
If you want to just test an already provided model, you should skip the training part and go to [Test section](#scrollTo=Y5jJf7v5Ox91)

# Training GeoWarp

In [7]:
!python3 /content/Team/GeoWarp/train.py --arch resnet50 --pooling gem --resume_fe /content/saved_models/cosface_model.pth

2022-12-27 17:24:30   python /content/Team/GeoWarp/train.py --arch resnet50 --pooling gem --resume_fe /content/saved_models/cosface_model.pth
2022-12-27 17:24:30   Arguments: Namespace(arch='resnet50', batch_size_consistency=16, batch_size_features_wise=16, batch_size_ss=16, channels=[225, 128, 128, 64, 64, 64, 64], consistency_w=0.1, dataset_name='sf-xs', datasets_folder='/content/small', exp_name='default', features_wise_w=10, iterations_per_epoch=500, k=0.6, kernel_sizes=[7, 5, 5, 5, 5, 5], lr=0.0001, n_epochs=3, num_reranked_preds=20, optim='sgd', pooling='gem', positive_dist_threshold=25, qp_num_workers=4, qp_threshold=1.2, resume_fe='/content/saved_models/cosface_model.pth', ss_num_workers=8, ss_w=1)
2022-12-27 17:24:30   The outputs are being saved in runs/default/2022-12-27_17-24-30
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' inste

# Training CosPlace



## Model

In [ ]:
from CosPlace import model
from model import network

model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
    logging.debug(f"Loading model from {args.resume_model}")
    model_state_dict = torch.load(args.resume_model)
    model.load_state_dict(model_state_dict)

model = model.to(args.device).train()

## Optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

## Datasets

### CosFace

In [ ]:
from CosPlace import datasets
from datasets.train_dataset import TrainDataset
from datasets.test_dataset import TestDataset
from Team.loss.cosface import CosFace

groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]

# apply cosface loss to each group
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [CosFace(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = TestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")

## Resume train

In [ ]:
from CosPlace import util

if args.resume_train:
    model, model_optimizer, classifiers, classifiers_optimizers, best_val_recall1, start_epoch_num = \
        util.resume_train(args, output_folder, model, model_optimizer, classifiers, classifiers_optimizers)
    model = model.to(args.device)
    epoch_num = start_epoch_num - 1
    logging.info(f"Resuming from epoch {start_epoch_num} with best R@1 {best_val_recall1:.1f} from checkpoint {args.resume_train}")
else:
    best_val_recall1 = start_epoch_num = 0

drive_tester(output_folder)

## Train and Evaluation Loop

In [ ]:
from CosPlace import augmentations, test

logging.info("Start training ...")
logging.info(f"There are {len(groups[0])} classes for the first group, " +
             f"each epoch has {args.iterations_per_epoch} iterations " +
             f"with batch_size {args.batch_size}, therefore the model sees each class (on average) " +
             f"{args.iterations_per_epoch * args.batch_size / len(groups[0]):.1f} times per epoch")


if args.augmentation_device == "cuda":
    gpu_augmentation = T.Compose([
            augmentations.DeviceAgnosticColorJitter(brightness=args.brightness,
                                                    contrast=args.contrast,
                                                    saturation=args.saturation,
                                                    hue=args.hue),
            augmentations.DeviceAgnosticRandomResizedCrop([512, 512],
                                                          scale=[1-args.random_resized_crop, 1]),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

if args.use_amp16:
    scaler = torch.cuda.amp.GradScaler()

for epoch_num in range(start_epoch_num, args.epochs_num):
    
    #### Train
    epoch_start_time = datetime.now()
    # Select classifier and dataloader according to epoch
    current_group_num = epoch_num % args.groups_num
    classifiers[current_group_num] = classifiers[current_group_num].to(args.device)
    util.move_to_device(classifiers_optimizers[current_group_num], args.device)
    
    dataloader = commons.InfiniteDataLoader(groups[current_group_num], num_workers=args.num_workers,
                                            batch_size=args.batch_size, shuffle=True,
                                            pin_memory=(args.device == "cuda"), drop_last=True)
    
    dataloader_iterator = iter(dataloader)
    model = model.train()
    
    epoch_losses = np.zeros((0, 1), dtype=np.float32)
    for iteration in tqdm(range(args.iterations_per_epoch), ncols=100):
        images, targets, _ = next(dataloader_iterator)
        images, targets = images.to(args.device), targets.to(args.device)
        
        if args.augmentation_device == "cuda":
            images = gpu_augmentation(images)
        
        model_optimizer.zero_grad()
        classifiers_optimizers[current_group_num].zero_grad()
        
        if not args.use_amp16:
            descriptors = model(images)
            output = classifiers[current_group_num](descriptors, targets)
            loss = criterion(output, targets)
            loss.backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            model_optimizer.step()
            classifiers_optimizers[current_group_num].step()
        else:  # Use AMP 16
            with torch.cuda.amp.autocast():
                descriptors = model(images)
                output = classifiers[current_group_num](descriptors, targets)
                loss = criterion(output, targets)
            scaler.scale(loss).backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            scaler.step(model_optimizer)
            scaler.step(classifiers_optimizers[current_group_num])
            scaler.update()
    
    classifiers[current_group_num] = classifiers[current_group_num].cpu()
    util.move_to_device(classifiers_optimizers[current_group_num], "cpu")
    
    logging.debug(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, "
                  f"loss = {epoch_losses.mean():.4f}")
    
    #### Evaluation
    recalls, recalls_str = test.test(args, val_ds, model)
    logging.info(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, {val_ds}: {recalls_str[:20]}")
    is_best = recalls[0] > best_val_recall1
    best_val_recall1 = max(recalls[0], best_val_recall1)
    # Save checkpoint, which contains all training parameters
    util.save_checkpoint({
        "epoch_num": epoch_num + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": model_optimizer.state_dict(),
        "classifiers_state_dict": [c.state_dict() for c in classifiers],
        "optimizers_state_dict": [c.state_dict() for c in classifiers_optimizers],
        "best_val_recall1": best_val_recall1
    }, is_best, output_folder)

    # save on drive
    drive_save_checkpoint(output_folder, start_time)


logging.info(f"Trained for {epoch_num+1:02d} epochs, in total in {str(datetime.now() - start_time)[:-7]}")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:root:Start training ...
INFO:root:There are 5965 classes for the first group, each epoch has 10000 iterations with batch_size 32, therefore the model sees each class (on average) 53.6 times per epoch
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|███████████████████████████████████████████████████████| 10000/10000 [1:14:13<00:00,  2.25it/s]
DEBUG:root:Epoch 00 in 1:14:14, loss = 7.9503
DEBUG:root:Extracting database descriptors for evaluation/te

# Test

## Import model

#### CosFace resume model path

In [9]:
resume_model = "/content/saved_models/cosface_model.pth"

In [10]:

from CosPlace import model, test, datasets
from model import network
from datasets.test_dataset import TestDataset

#### Model
model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
  if os.path.exists(args.resume_model):
    resume_model = args.resume_model

if resume_model is not None:
  if not os.path.exists(resume_model):
    raise FileNotFoundError(f"Model {resume_model} not found.")

  logging.info(f"Loading model from {resume_model}")
  model_state_dict = torch.load(resume_model)
  model.load_state_dict(model_state_dict)
else:
    logging.info("WARNING: You didn't provide a path to resume the model (--resume_model parameter). " +
                 "Evaluation will be computed using randomly initialized weights.")

model = model.to(args.device)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

DEBUG:root:Train only layer3 and layer4 of the resnet18, freeze the previous ones
INFO:root:There are 1 GPUs and 2 CPUs.
INFO:root:Loading model from /content/saved_models/cosface_model.pth


## Test on SF-XS

Test the model on the sf-xs (test) dataset. WITHOUT GeoWarp

In [ ]:
# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|███████████████████████████████████████████████████████████| 1700/1700 [03:52<00:00,  7.32it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|███████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 62.62it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 1000; #db: 27191 >: R@1: 50.4, R@5: 64.4, R@10: 68.9, R@20: 73.5


Test the model on the sf-xs (test) dataset. WITH GeoWarp

In [12]:
from Team import test_with_geowarp as testgeo

# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = testgeo.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  1%|▉                                                            | 25/1700 [00:11<13:01,  2.14it/s]
Exception in thread Thread-23:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _p

KeyboardInterrupt: ignored

## Test on Tokyo-XS

Test the model on the tokyo-xs dataset. WITHOUT GeoWarp

In [ ]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████████████████████| 799/799 [02:02<00:00,  6.55it/s]
DEBUG:root:Extracting queries descriptors for evaluation/testing using batch size 1
100%|█████████████████████████████████████████████████████████████| 315/315 [00:05<00:00, 58.72it/s]
DEBUG:root:Calculating recalls
INFO:root:< test - #q: 315; #db: 12771 >: R@1: 70.8, R@5: 85.7, R@10: 90.5, R@20: 93.0


Test the model on the tokyo-xs dataset WITH GeoWarp

In [14]:
from Team import test_with_geowarp as testgeo

tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = testgeo.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  1%|▍                                                              | 6/799 [00:03<06:52,  1.92it/s]


KeyboardInterrupt: ignored

## Test on Tokyo-Night

Test the model on the tokyo-night dataset WITHOUT GeoWarp.

In [15]:
from Team import test_with_geowarp as testgeo

tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = testgeo.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

DEBUG:root:Extracting database descriptors for evaluation/testing
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  2%|█▍                                                            | 18/799 [00:04<03:21,  3.87it/s]


KeyboardInterrupt: ignored

Test the model on the tokyo-night dataset WITH GeoWarp

In [ ]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

# Save results

## Create CSV with recalls

In [ ]:
import csv
header = ["sf-xs (test)", "Tokyo-xs", "Tokyo-night"]
data = [sf_xs_r15, tokyo_xs_r15, tokyo_night_r15]

for h, d in zip(header, data):
  logging.info(f"{h}: {d}")

with open(f"/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv", "w") as f:
  writer = csv.writer(f)

  writer.writerow(header)
  writer.writerow(data)
logging.info(f"save table results to /content/{SAVEDIR}{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv")

INFO:root:sf-xs (test): 50.4/64.4
INFO:root:Tokyo-xs: 70.8/85.7
INFO:root:Tokyo-night: 54.3/72.4
INFO:root:save table results to /content/sphereface2022-12-16_21-36-11.csv


## Save on Drive

#### CosFace

In [ ]:
modelLocal = "cosface_model.pth"

Save all data generated by this notebook in a specific folder in **PERSONAL** gdrive. Remember to copy inside shared_data of project drive

### Run the following if you did training

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

if not os.path.exists("/content/drive/MyDrive/step_3"):
  !mkdir /content/drive/MyDrive/step_3
!cp "/content/logs/{SAVEDIR}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}/best_model.pth" "/content/drive/MyDrive/step_3/{modelLocal}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/logs/sphereface/2022-12-16_21-36-11/best_model.pth': No such file or directory


### Run always

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

# zip logs -> logs.zip
!zip -r /content/drive/MyDrive/step_3/logs.zip /content/logs/
# zip cache -> cache.zip
if os.path.exists("/content/cache/"):
  !zip -r /content/drive/MyDrive/step_3/cache.zip /content/cache/

!cp "/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv" /content/drive/MyDrive/step_3/

drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
updating: content/logs/ (stored 0%)
updating: content/logs/arcface/ (stored 0%)
updating: content/logs/arcface/2022-12-16_21-06-49/ (stored 0%)
updating: content/logs/arcface/2022-12-16_21-06-49/debug.log (deflated 76%)
updating: content/logs/arcface/2022-12-16_21-06-49/info.log (deflated 74%)
  adding: content/logs/sphereface/ (stored 0%)
  adding: content/logs/sphereface/2022-12-16_21-36-11/ (stored 0%)
  adding: content/logs/sphereface/2022-12-16_21-36-11/debug.log (deflated 61%)
  adding: content/logs/sphereface/2022-12-16_21-36-11/info.log (deflated 53%)
